In [20]:
from pandas import DataFrame, date_range, read_table, to_datetime
from datetime import datetime, timedelta, date
from collections import defaultdict
from itertools import product
import numpy as np

In [38]:
path = '/home/diego/Documents/Salidas/'
file_names = {'ClinicaSantaRosa-HospitalUniversitario':'datos_jardin.csv',
             'Marengo_1':'datos_tibaitata.csv',
             'EBTRF_Villavicencio':'datos_icavill.csv',
             'MuseoVillaDeLeyva':'datos_villa.csv'}
obs = {'ClinicaSantaRosa-HospitalUniversitario':None,
             'Marengo_1':None,
             'EBTRF_Villavicencio':None,
             'MuseoVillaDeLeyva':None}
dates = date_range(start='2019-11-01', end='2020-01-15')

#Open observation files
for file in file_names.keys():
    obs[file] = read_table('{}{}'.format(path, file_names[file]), sep=',')
    
#Open forecast files
forecast = {0:None, 1:None, 2:None}
for file in (0, 1, 2):
    forecast[file] = read_table('{}Prec_{}.csv'.format(path, file), sep=',')
    forecast[file]['Date'] = to_datetime(forecast[file]['Date'])
    forecast[file].index = forecast[file]['Date']

In [57]:
date

Timestamp('2019-11-01 00:00:00', freq='D')

In [56]:
# Hourly forecast to daily forecast
forecast_daily = {0:None, 1:None, 2:None}

for forday, date in product((0, 1, 2), dates):
    if date == dates[0]:
        df = DataFrame(columns=forecast[0].columns)
    day = datetime.strptime(date, '%Y/%m/%d')
    nextday = datetime.strptime(date + timedelta(days=1), '%Y/%m/%d')
    row = forecast[forday].loc['{} 07:00:00':'{} 07:00:00'.format(
                    day, nextday)].sum()
    row.name = day
    df.append(row)
    if day == dates[-1]:
        forecast_daily = df

TypeError: strptime() argument 1 must be str, not Timestamp

In [53]:
obj = forecast[0].loc['2019-11-01 07:00:00':'2019-11-02 07:00:00'].sum()
obj.name = '2019-11-01'
df = DataFrame(columns=forecast[0].columns)
df.append(obj, ignore_index=False)

,Date,Marengo_1,Marengo_2,Marengo_3,ClinicaSantaRosa-HospitalUniversitario,Campus-UrielGutierrez-CamiloTorres,Casa_Calle60,Casa_Gaitan,CerroAltoDelCable,EdificioLasNieves,ClaustroSanAgustin-Observatorio,EBTRF_Villavicencio,LaEsperanza-Guaymaral,MuseoVillaDeLeyva
2019-11-01,NaN,5.0,9.0,9.0,21.0,23.0,23.0,21.0,25.0,19.0,33.0,0.0,7.0,10.0
